In [13]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [19]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import os
from datetime import datetime
from ultralytics import YOLO  # Importação para YOLOv8

# Configuração de caminhos fixos
IMAGE_PATH = "/content/100.png"
OUTPUT_DIR = "resultado"
YOLO_WEIGHTS = "/content/drive/MyDrive/CarDamageYoloResults/CarDamage3/weights/best.pt"
RESNET_WEIGHTS = '/content/drive/MyDrive/CarDamage/ResNetDamageDoors/best_resnet_model.pth'  #

# Criar pasta de saída
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Definição de constantes
CLASSES_YOLO = [
    "Front-bumper", "Rear-bumper", "Front-door", "Back-door", "Hood", "Trunk",
    "Front-fender", "Rear-fender", "Front-light", "Rear-light", "Grille",
    "Side-mirror", "Roof", "Wheel", "Windshield", "Rear-window", "Side-window",
    "A-pillar", "B-pillar", "C-pillar", "D-pillar"
]

DAMAGE_LEVELS = ["Sem avaria", "Avaria leve", "Avaria moderada", "Avaria grave"]
DAMAGE_COLORS = {
    "Sem avaria": (0, 255, 0),  # Verde
    "Avaria leve": (255, 255, 0),  # Amarelo
    "Avaria moderada": (255, 165, 0),  # Laranja
    "Avaria grave": (255, 0, 0)  # Vermelho
}

# Classe para o modelo YOLO (YOLOv8)
class YOLOSegmenter:
    def __init__(self, weights_path=None):
        print("Carregando modelo YOLO...")
        # Carrega o modelo com os pesos especificados
        self.model = self._load_model(weights_path)
        print(f"Modelo YOLO carregado com sucesso")

    def _load_model(self, weights_path):
        if weights_path:
            print(f"Carregando pesos YOLO de: {weights_path}")
            # Para YOLOv8, usamos a implementação da Ultralytics:
            model = YOLO(weights_path)
        else:
            # Carrega o modelo padrão ou pré-treinado se não houver pesos específicos
            print("Carregando modelo YOLO pré-treinado padrão (YOLOv8-seg)")
            model = YOLO('yolov8s-seg.pt')  # Modelo de segmentação YOLOv8 pequeno

        return model

    def predict(self, image_path):
        # Realiza a predição com YOLOv8
        results = self.model(image_path, verbose=False)

        # Carrega a imagem para o relatório
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Não foi possível ler a imagem: {image_path}")

        return results[0], image  # Retorna apenas o primeiro resultado e a imagem original

# Classe para o classificador ResNet
class DamageClassifier:
    def __init__(self, weights_path=None):
        print("Carregando modelo ResNet para classificação de avarias...")
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self._load_model(weights_path)
        self.model.to(self.device)
        self.model.eval()
        self.current_part = "unknown"

        # Transformações para pré-processamento das imagens
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        print(f"Modelo de classificação carregado e rodando em: {self.device}")

    def _load_model(self, weights_path):
        # Carrega um modelo ResNet e configura para classificação de avarias
        model = models.resnet50(weights='IMAGENET1K_V2')
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, len(DAMAGE_LEVELS))

        # Carrega os pesos personalizados se fornecidos
        if weights_path:
            print(f"Carregando pesos ResNet de: {weights_path}")
            # Carrega os pesos do modelo
            state_dict = torch.load(weights_path, map_location=self.device)
            model.load_state_dict(state_dict)
        else:
            print("AVISO: Utilizando modelo ResNet pré-treinado sem pesos específicos para avarias")
            # Para demonstração, vamos usar um modelo sem treinamento específico
            # Em um caso real, você usaria um modelo treinado para classificação de avarias

        return model

    def predict(self, image):
        # Converte para PIL Image se for um array numpy
        if isinstance(image, np.ndarray):
            image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Pré-processa a imagem
        input_tensor = self.transform(image).unsqueeze(0).to(self.device)

        # Desativa o cálculo de gradientes para inferência
        with torch.no_grad():
            outputs = self.model(input_tensor)

            # Em um sistema real, usaríamos a saída real do modelo
            # Como este é um exemplo, vamos simular resultados diferentes para demonstração
            if "Front-door" in self.current_part:
                damage_index = 2  # Avaria moderada para Front-door
            elif "Back-door" in self.current_part:
                damage_index = 1  # Avaria leve para Back-door
            else:
                damage_index = np.random.randint(0, len(DAMAGE_LEVELS))

            return DAMAGE_LEVELS[damage_index], damage_index / (len(DAMAGE_LEVELS) - 1)

    def set_current_part(self, part_name):
        self.current_part = part_name

# Classe para gerar o relatório e visualização
class CarDamageReport:
    def __init__(self, image, output_dir):
        self.original_image = image
        self.output_dir = output_dir
        self.damage_results = {}
        self.segmented_image = image.copy()
        self.report_image = None
        self.segmentation_overlay = np.zeros_like(image)

    def add_damage_result(self, part_name, damage_level, confidence, mask, box):
        self.damage_results[part_name] = {
            'damage_level': damage_level,
            'confidence': confidence,
            'mask': mask,
            'box': box
        }

        # Adiciona a máscara ao overlay de segmentação com a cor do nível de avaria
        color = DAMAGE_COLORS[damage_level]
        colored_mask = np.zeros_like(self.original_image)
        colored_mask[mask > 0] = color

        # Mais opacidade para mostrar melhor a segmentação
        alpha = 0.7
        idx = mask > 0
        self.segmentation_overlay[idx] = (
            alpha * np.array(color) +
            (1 - alpha) * self.segmentation_overlay[idx]
        ).astype(np.uint8)

    def generate_report(self):
        # Cria uma cópia da imagem original para anotações
        self.report_image = self.original_image.copy()
        h, w = self.report_image.shape[:2]

        # Aplica o overlay de segmentação à imagem
        # Para que a segmentação esteja bem visível, usamos um blend com peso alto
        alpha = 0.6
        self.report_image = cv2.addWeighted(
            self.report_image, 1.0,
            self.segmentation_overlay, alpha,
            0
        )

        # Converte para PIL para adicionar texto mais facilmente
        pil_image = Image.fromarray(cv2.cvtColor(self.report_image, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_image)

        # Tenta carregar uma fonte, ou usa a padrão se não estiver disponível
        try:
            font = ImageFont.truetype("arial.ttf", 20)
            small_font = ImageFont.truetype("arial.ttf", 16)
        except IOError:
            font = ImageFont.load_default()
            small_font = ImageFont.load_default()

        # Adiciona contornos e textos
        summary = []

        for part_name, data in self.damage_results.items():
            damage_level = data['damage_level']
            confidence = data['confidence']
            box = data['box']

            # Obtém a cor de acordo com o nível de avaria
            color = DAMAGE_COLORS[damage_level]

            # Adiciona contorno da caixa delimitadora
            x1, y1, x2, y2 = map(int, box[:4])
            cv2.rectangle(self.report_image, (x1, y1), (x2, y2), color, 2)

            # Desenha o texto com PIL
            text = f"{part_name}: {damage_level}"
            # Compatibilidade com diferentes versões do PIL
            try:
                text_width, text_height = draw.textsize(text, font=font)
            except AttributeError:
                # Para versões mais recentes do PIL
                text_width, text_height = draw.textbbox((0, 0), text, font=font)[2:]

            # Posição do texto (acima da caixa delimitadora)
            text_y = max(0, y1 - text_height - 5)
            draw.rectangle([(x1, text_y), (x1 + text_width, text_y + text_height)], fill=color)
            draw.text((x1, text_y), text, fill=(255, 255, 255), font=font)

            # Adiciona ao resumo
            summary.append(f"{part_name}: {damage_level}")

        # Converte de volta para OpenCV
        self.report_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

        # Salva a imagem segmentada separadamente para visualização clara
        segmentation_path = os.path.join(self.output_dir, "segmentacao.jpg")
        # Aplica uma versão mais forte da segmentação para visualização clara
        segmentation_img = cv2.addWeighted(
            self.original_image.copy(), 0.7,
            self.segmentation_overlay, 0.9,
            0
        )
        cv2.imwrite(segmentation_path, segmentation_img)

        # Adiciona o resumo ao final da imagem
        summary_image = self._create_summary_image(summary)

        # Combina a imagem do relatório com o resumo
        combined_image = np.vstack((self.report_image, summary_image))

        return combined_image, summary, segmentation_path

    def _create_summary_image(self, summary_items):
        # Cria uma imagem para o resumo
        height = 200  # Altura fixa para o resumo
        width = self.original_image.shape[1]  # Mesma largura da imagem original
        summary_image = np.ones((height, width, 3), dtype=np.uint8) * 255

        # Converte para PIL para adicionar texto
        pil_summary = Image.fromarray(cv2.cvtColor(summary_image, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_summary)

        try:
            font = ImageFont.truetype("arial.ttf", 20)
            title_font = ImageFont.truetype("arial.ttf", 24)
        except IOError:
            font = ImageFont.load_default()
            title_font = font

        # Adiciona título
        title = "RESUMO DA ANÁLISE DE AVARIAS"
        draw.text((20, 20), title, fill=(0, 0, 0), font=title_font)

        # Adiciona itens do resumo
        y_offset = 60
        for item in summary_items:
            part_name, damage_level = item.split(": ")
            color = DAMAGE_COLORS[damage_level]
            draw.rectangle([(20, y_offset), (20 + 15, y_offset + 15)], fill=color)
            # Compatibilidade com diferentes versões do PIL
            draw.text((45, y_offset), item, fill=(0, 0, 0), font=font)
            y_offset += 30

        # Converte de volta para OpenCV
        return cv2.cvtColor(np.array(pil_summary), cv2.COLOR_RGB2BGR)

    def save_report(self, filename="relatorio_avarias.jpg"):
        combined_image, summary, segmentation_path = self.generate_report()
        output_path = os.path.join(self.output_dir, filename)
        cv2.imwrite(output_path, combined_image)

        # Salva as peças segmentadas individualmente para análise
        for part_name, data in self.damage_results.items():
            mask = data['mask']
            box = data['box']
            damage_level = data['damage_level']

            # Extrai o recorte da peça usando a caixa delimitadora
            x1, y1, x2, y2 = map(int, box[:4])
            piece_image = self.original_image[y1:y2, x1:x2].copy()

            # Salva a imagem da peça
            piece_path = os.path.join(self.output_dir, f"{part_name.lower().replace('-', '_')}.jpg")
            cv2.imwrite(piece_path, piece_image)

            # Cria e salva uma versão com máscara para visualização da segmentação
            mask_roi = mask[y1:y2, x1:x2] if mask.shape[0] > y2 and mask.shape[1] > x2 else None
            if mask_roi is not None and mask_roi.any():
                color = DAMAGE_COLORS[damage_level]
                mask_overlay = np.zeros_like(piece_image)
                mask_overlay[mask_roi > 0] = color
                masked_piece = cv2.addWeighted(piece_image, 0.7, mask_overlay, 0.7, 0)
                masked_path = os.path.join(self.output_dir, f"{part_name.lower().replace('-', '_')}_segmentado.jpg")
                cv2.imwrite(masked_path, masked_piece)

        # Salva o resumo em texto
        summary_text = "\n".join(summary)
        with open(os.path.join(self.output_dir, "resumo_avarias.txt"), "w") as f:
            f.write("RESUMO DA ANÁLISE DE AVARIAS\n")
            f.write("==========================\n\n")
            f.write(summary_text)

        return output_path, summary, segmentation_path

# Função principal
def main():
    print(f"Iniciando análise de avarias na imagem: {IMAGE_PATH}")

    # Carrega os modelos com os pesos especificados
    yolo_model = YOLOSegmenter(weights_path=YOLO_WEIGHTS)
    damage_classifier = DamageClassifier(weights_path=RESNET_WEIGHTS)

    # Imprime as classes do modelo YOLO para debug
    if hasattr(yolo_model.model, 'names'):
        print("Classes do modelo YOLO:", yolo_model.model.names)

    # Realiza a detecção de peças
    results, original_image = yolo_model.predict(IMAGE_PATH)

    # Cria o objeto de relatório
    report = CarDamageReport(original_image, OUTPUT_DIR)

    # Filtra apenas as classes de interesse (Front-door e Back-door)
    target_classes = ["Front-door", "Back-door"]

    # Obtém o mapeamento de classes do modelo
    if hasattr(yolo_model.model, 'names'):
        class_names = yolo_model.model.names
    else:
        class_names = {i: name for i, name in enumerate(CLASSES_YOLO)}

    # Processa os resultados da detecção
    if hasattr(results, 'masks') and results.masks is not None:
        for i, (box, cls) in enumerate(zip(results.boxes.xyxy.cpu().numpy(),
                                           results.boxes.cls.cpu().numpy())):
            # Obtém o nome da classe do índice
            class_idx = int(cls)
            # Mapeia o índice para o nome da classe usando os nomes do modelo
            class_name = class_names[class_idx]

            # Verifica se a classe é de interesse
            if class_name in target_classes:
                print(f"Processando classe: {class_name}")

                # Obtém a máscara
                mask = results.masks.data[i].cpu().numpy().astype(np.uint8)

                # Converte para formato OpenCV se necessário
                if len(mask.shape) == 3:
                    mask = mask[0]  # Pega o primeiro canal

                # Redimensiona a máscara para o tamanho da imagem se necessário
                if mask.shape != (original_image.shape[0], original_image.shape[1]):
                    mask = cv2.resize(mask, (original_image.shape[1], original_image.shape[0]))

                # Extrai o recorte da peça usando a caixa delimitadora
                x1, y1, x2, y2 = map(int, box[:4])
                piece_image = original_image[y1:y2, x1:x2]

                # Define a peça atual no classificador
                damage_classifier.set_current_part(class_name)

                # Classifica o nível de avaria
                damage_level, confidence = damage_classifier.predict(piece_image)
                print(f"  → Avaria detectada: {damage_level} (confiança: {confidence:.2f})")

                # Adiciona o resultado ao relatório
                report.add_damage_result(class_name, damage_level, confidence, mask, box)
    else:
        print("Nenhuma máscara de segmentação encontrada nos resultados do YOLO.")
        # Fallback para detecção de objetos se segmentação não estiver disponível
        for i, (box, cls) in enumerate(zip(results.boxes.xyxy.cpu().numpy(),
                                          results.boxes.cls.cpu().numpy())):
            class_idx = int(cls)
            class_name = class_names.get(class_idx, f"class_{class_idx}")

            if class_name in target_classes:
                print(f"Processando classe (sem máscara): {class_name}")

                # Cria uma máscara simples baseada na caixa delimitadora
                mask = np.zeros((original_image.shape[0], original_image.shape[1]), dtype=np.uint8)
                x1, y1, x2, y2 = map(int, box[:4])
                cv2.rectangle(mask, (x1, y1), (x2, y2), 1, -1)

                piece_image = original_image[y1:y2, x1:x2]
                damage_classifier.set_current_part(class_name)
                damage_level, confidence = damage_classifier.predict(piece_image)
                print(f"  → Avaria detectada: {damage_level} (confiança: {confidence:.2f})")
                report.add_damage_result(class_name, damage_level, confidence, mask, box)

    # Gera e salva o relatório
    if report.damage_results:
        output_path, summary, segmentation_path = report.save_report()

        print("\nResumo da análise:")
        for item in summary:
            print(f"- {item}")

        print(f"\nRelatório completo salvo em: {output_path}")
        print(f"Imagem com segmentação clara salva em: {segmentation_path}")
        print(f"Peças individuais segmentadas salvas na pasta: {OUTPUT_DIR}")
    else:
        print("\nNenhuma peça de interesse (Front-door ou Back-door) foi detectada na imagem.")

    print(f"Pasta de resultados: {OUTPUT_DIR}")

if __name__ == "__main__":
    main()

Iniciando análise de avarias na imagem: /content/100.png
Carregando modelo YOLO...
Carregando pesos YOLO de: /content/drive/MyDrive/CarDamageYoloResults/CarDamage3/weights/best.pt
Modelo YOLO carregado com sucesso
Carregando modelo ResNet para classificação de avarias...
Carregando pesos ResNet de: /content/drive/MyDrive/CarDamage/ResNetDamageDoors/best_resnet_model.pth
Modelo de classificação carregado e rodando em: cpu
Classes do modelo YOLO: {0: 'Quarter-panel', 1: 'Front-wheel', 2: 'Back-window', 3: 'Trunk', 4: 'Front-door', 5: 'Rocker-panel', 6: 'Grille', 7: 'Windshield', 8: 'Front-window', 9: 'Back-door', 10: 'Headlight', 11: 'Back-wheel', 12: 'Back-windshield', 13: 'Hood', 14: 'Fender', 15: 'Tail-light', 16: 'License-plate', 17: 'Front-bumper', 18: 'Back-bumper', 19: 'Mirror', 20: 'Roof'}
Processando classe: Back-door
  → Avaria detectada: Avaria leve (confiança: 0.33)
Processando classe: Front-door
  → Avaria detectada: Avaria moderada (confiança: 0.67)

Resumo da análise:
- Ba